In [ ]:
!pip install biopython

In [ ]:
import csv
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from collections import Counter
from math import log2

# Function to calculate sequence complexity (Shannon entropy)
def sequence_complexity(epitope):
    counts = Counter(epitope)
    total = len(epitope)
    entropy = -sum((count/total) * log2(count/total) for count in counts.values())
    return entropy

# Function to count potential phosphorylation sites (S, T, Y)
def count_phosphorylation_sites(epitope):
    return epitope.count('S') + epitope.count('T') + epitope.count('Y')

# Function to count potential glycosylation sites (N-X-S/T)
def count_glycosylation_sites(epitope):
    count = 0
    for i in range(len(epitope) - 2):
        if epitope[i] == 'N' and epitope[i+2] in 'ST' and epitope[i+1] != 'P':
            count += 1
    return count

# Function to predict disulfide bonds (basic heuristic, not highly accurate)
def predict_disulfide_bonds(epitope):
    return epitope.count('C') // 2

# Function to extract features from an epitope
def extract_epitope_features(epitope):
    if len(epitope) == 0:
        return {}
    analyzed_seq = ProteinAnalysis(epitope)
    
    features = {
        "epitope": epitope,
        "length": len(epitope),
        "aa_composition": dict(Counter(epitope)),
        "percentage_composition": {aa: count / len(epitope) * 100 for aa, count in Counter(epitope).items()},
        "molecular_weight": analyzed_seq.molecular_weight(),
        "aromaticity": analyzed_seq.aromaticity(),
        "instability_index": analyzed_seq.instability_index(),
        "hydrophobicity": analyzed_seq.gravy(),
        "isoelectric_point": analyzed_seq.isoelectric_point(),
        "extinction_coefficient": analyzed_seq.molar_extinction_coefficient()[0],  # assuming reduced Cys
        "secondary_structure_fraction": analyzed_seq.secondary_structure_fraction(),
        "flexibility": analyzed_seq.flexibility(),
        "charge_at_pH7": analyzed_seq.charge_at_pH(7.0),
        "disulfide_bonds": predict_disulfide_bonds(epitope),
        "phosphorylation_sites": count_phosphorylation_sites(epitope),
        "glycosylation_sites": count_glycosylation_sites(epitope),
        "sequence_complexity": sequence_complexity(epitope)
    }
    return features

def flatten_features(features):
    if not features:
        return {}
    
    flexibility_list = features.get("flexibility", [])
    if flexibility_list:
        flexibility_avg = sum(flexibility_list) / len(flexibility_list)
    else:
        flexibility_avg = 0  # Or use another default value
    
    flattened = {
        "epitope": features["epitope"],
        "length": features["length"],
        "molecular_weight": features["molecular_weight"],
        "aromaticity": features["aromaticity"],
        "instability_index": features["instability_index"],
        "hydrophobicity": features["hydrophobicity"],
        "isoelectric_point": features["isoelectric_point"],
        "extinction_coefficient": features["extinction_coefficient"],
        "helix_fraction": features["secondary_structure_fraction"][0],
        "turn_fraction": features["secondary_structure_fraction"][1],
        "sheet_fraction": features["secondary_structure_fraction"][2],
        "flexibility": flexibility_avg,
        "charge_at_pH7": features["charge_at_pH7"],
        "disulfide_bonds": features["disulfide_bonds"],
        "phosphorylation_sites": features["phosphorylation_sites"],
        "glycosylation_sites": features["glycosylation_sites"],
        "sequence_complexity": features["sequence_complexity"]
    }
    
    # Add amino acid composition and percentage composition
    for aa, count in features["aa_composition"].items():
        flattened[f"aa_{aa}_count"] = count
    
    for aa, percentage in features["percentage_composition"].items():
        flattened[f"aa_{aa}_percentage"] = percentage
    
    return flattened

# File paths
input_file = "epitopes.txt"
output_file = "epitope_features.csv"

# Read epitopes from the text file
with open(input_file, "r") as f:
    epitopes = f.read().splitlines()

# Open CSV file for writing
with open(output_file, "w", newline="") as csvfile:
    # Define CSV column headers
    fieldnames = [
        "epitope", "length", "molecular_weight", "aromaticity", 
        "instability_index", "hydrophobicity", 
        "isoelectric_point", "extinction_coefficient", 
        "helix_fraction", "turn_fraction", "sheet_fraction", 
        "flexibility", "charge_at_pH7", "disulfide_bonds",
        "phosphorylation_sites", "glycosylation_sites", 
        "sequence_complexity"
    ]
    
    # Adding amino acid columns dynamically based on the dataset
    all_amino_acids = {f"aa_{aa}_count" for epitope in epitopes for aa in set(epitope)}
    all_amino_acids_percentage = {f"aa_{aa}_percentage" for epitope in epitopes for aa in set(epitope)}
    
    fieldnames.extend(sorted(all_amino_acids))
    fieldnames.extend(sorted(all_amino_acids_percentage))

    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    # Process each epitope and write to the CSV
    for epitope in epitopes:
        features = extract_epitope_features(epitope)
        flattened_features = flatten_features(features)

        # Fill empty entries with 0
        for field in fieldnames:
            if field not in flattened_features:
                flattened_features[field] = 0
                
        writer.writerow(flattened_features)
    
print(f"Feature extraction completed. Data saved to {output_file}")